# Reinforcement Learning in Finite MDPs

In [0]:
!git clone https://github.com/rlgammazero/mvarl_hands_on.git > /dev/null 2>&1

## MDPs

In [0]:
import sys
sys.path.insert(0, './mvarl_hands_on/utils')
import numpy as np
from scipy.special import softmax # for SARSA
import matplotlib.pyplot as plt
import json
import math
from cliffwalk import CliffWalk
#from test_env import ToyEnv1

Setting up the environment

In [0]:
env = CliffWalk(proba_succ=0.98)

####################################################################################
# You probably want to test smaller enviroments before
#env = ToyEnv1(gamma=0.95)
####################################################################################

# Useful attributes
print("Set of states:", env.states)
print("Set of actions:", env.actions)
print("Number of states: ", env.Ns)
print("Number of actions: ", env.Na)
print("P has shape: ", env.P.shape)  # P[s, a, s'] = env.P[s, a, s']
print("discount factor: ", env.gamma)
print("")

# Usefult methods
state = env.reset() # get initial state
print("initial state: ", state)
print("reward at (s=1, a=3,s'=2): ", env.reward_func(1,3,2))
print("")

# A random policy
policy = np.random.randint(env.Na, size = (env.Ns,))
print("random policy = ", policy)

# Interacting with the environment
print("(s, a, s', r):")
for time in range(4):
    action = policy[state]
    next_state, reward, done, info = env.step(action)
    print(state, action, next_state, reward)
    if done:
        break
    state = next_state
print("")
print(env.R.shape)

## Question 1: Value iteration
1. Write a function applying the optimal Bellman operator on a provided Q function: $Q_1 = LQ_0, \; Q_0\in \mathbb{R}^{S\times A}$
2. Write a function implementing Value Iteration (VI) with $\infty$-norm stopping condition (reuse function implemented in 1)
3. Evaluate the convergence of your estimate, i.e., plot the value $\|Q_n - Q^\star\|_{\infty} = \max_{s,a} |Q_n(s,a) - Q^\star(s,a)|$

In [0]:
# --------------
# Point 1
# --------------
def bellman_operator(Q0, Ns, Na, R, P, gamma):
    Q1 = np.zeros((Ns,Na))
    for s in range(Ns):
        for a in range(Na):
            Q1[s,a] = np.dot(P[s,a,:],R[s,a,:]) + gamma*np.sum(P[s,a,:]*np.max(Q0, 1))
            
    greedy_policy = np.argmax(Q1, axis=1)
    
    return Q1, greedy_policy

In [0]:
# --------------
# Point 2
# --------------
def value_iteration(Q0, env, max_iter, epsilon=1e-5):
    Q_history = []
    Q_history.append(Q0)
    
    P = env.P
    Ns = env.Ns
    Na = env.Na
    R = env.R
    gamma = env.gamma

    Q1 = np.zeros((Ns,Na))
    Q1[:] = Q0[:]
    Q2 = np.zeros((Ns, Na))
    residuals = np.zeros((max_iter))
    for i in range(max_iter):
        
        Q2 , greedy_policy = bellman_operator(Q1, Ns, Na, R, P, gamma)
        residuals[i] = np.max(np.abs(Q2-Q1))
        
        if residuals[i]<epsilon:
            residuals = residuals[:i+1]
            break
        
        Q1[:] = Q2[:]
        
        Q_history.append(Q2)
    Q = Q2
    return Q, greedy_policy, Q_history

In [0]:
# --------------
# Point 3
# --------------
with open("./mvarl_hands_on/data/Q_opts.json", "r") as fp:
    Qopts = json.load(fp)
Qstar = Qopts["{}_{}".format(type(env).__name__,env.gamma)]

Ns = env.Ns
Na = env.Na

Q0 = np.zeros((Ns,Na))
max_iter = 1500
Q, greedy_policy, Q_history = value_iteration(Q0,env,max_iter)

norm_values = [np.max(np.abs(Q_history[i] - Q_history[-1])) for i in range(len(Q_history)-1)]

plt.plot(norm_values)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Q-learning: Convergence of Q")

In [0]:
state = env.reset()
env.render()
for i in range(50):
    action = greedy_policy[state]
    state, reward, done, _ = env.step(action)
    env.render()

## Question 2: Q learning
Q learning is a model-free algorithm for estimating the optimal Q-function online.
It is an off-policy algorithm since the samples are collected with a policy that is (potentially) not the one associated to the estimated Q-function.

1. Implement Q learning with $\epsilon$-greedy exploration.
  - Plot the error in Q-functions over iterations
  - Plot the sum of rewards as a function of iteration


$\epsilon$-greedy policy:
$$
\pi(s) = \begin{cases}
\max_a Q(s,a) & \text{w.p.} \epsilon\\
\text{random action} & \text{w.p.} 1- \epsilon
\end{cases}
$$

In [0]:
# ---------------------------
# Q-Learning
# ---------------------------
def epsilon_greedy(Q, s, epsilon):
    a = np.argmax(Q[s,:])
    if(np.random.rand()<=epsilon): # random action
        aa = np.random.randint(Na-1)
        if aa==a:
            a=Na-1
        else: 
            a=aa
    return a


def Qlearning(Q0, max_steps, Q_opt):
    Qql = Q0
    norm_values = []
    t = 1
    alpha = 1
    epsilon = 1
    x = env.reset()
    while t < max_steps:
        if(t%10000==0):
            epsilon = epsilon/2
        a = epsilon_greedy(Qql,x,epsilon)
        y,r,d,_ = env.step(a)
        alpha = 1/(t**(1/24))
        Qql[x][a] = Qql[x][a] + alpha * (r + env.gamma * np.max(Qql[y][:]) - Qql[x][a])
        norm_values.append(np.abs(Qql - Q_opt).mean())
        if d==True:
            x = env.reset()
        else:
            x=y
        t = t + 1
        
    return Qql, norm_values



In the section above you have an implementation of Q-learning. The step size $\alpha_t$ is set to $\alpha_t = \frac{1}{t^{1/24}}$. From a theoretical point of view, this is not a good choice, because this step size does not respect the robbins-monroe conditions. However, this heuristic seems to work well in practice. \

Moreover, the exploration-exploitation dilemma is done through the update of the variable epsilon. I have decided to devide epsilon by two after a certain number of time steps. A better way of doing Q-learning would be to consider epsilon as a function of a state and to adapt the update of epsilon given the number of times this state has been visited before.

In [0]:
# --------------
# Point 1
# --------------
# Number of Q learning steps
max_steps = int(1e5)  
# max_steps = 10

Q0 = np.zeros((env.Ns, env.Na))
# Use the previous code to verify the correctness of q learning
Q_opt, pi_opt, Q_hist = value_iteration(Q0, env, 2000, epsilon=1e-8)

Qql, norm_values = Qlearning(Q0, max_steps, Q_opt)
#Qql2, norm_values = Qlearning(Qql, max_steps, Q_opt)
#Qql3, norm_values = Qlearning(Qql2, max_steps, Q_opt)

print(env.render())
print("optimal policy: ", pi_opt)
greedy_policy = np.argmax(Qql, axis=1)
#greedy_policy = np.argmax(Qql2, axis=1)
#greedy_policy = np.argmax(Qql3, axis=1)
print("est policy:", greedy_policy)

plt.plot(norm_values)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Q-learning: Convergence of Q")
plt.show()

# how confident are you in the performance of the algorithm? maybe a single run is not enough

To run the Q-learning algorithm several times, please uncomment the corresponding lines. I did it and it did improve the performances. But we are still very far from the results obtained in part 1. As I explained before, a better way to diminish the error would be to do a "smarter" exploration of the environment. 